# **Behavioral Cloning Writeup Report**

## Behavioral Cloning Project

The goals / steps of this project are the following:

- Use the simulator to collect data of good driving behavior
- Build, a convolution neural network in Keras that predicts steering angles from images
- Train and validate the model with a training and validation set
- Test that the model successfully drives around track one without leaving the road
- Summarize the results with a written report


## Files Submitted & Code Quality

### 1. Submission includes all required files and can be used to run the simulator in autonomous mode

My project includes the following files:

- model_nvidia_prep_lin_test.ipynb - containing the script to create and train the model
- drive.py - for driving the car in autonomous mode
- model_nvidia_prep_lin_test.h5 - containing a trained convolution neural network
- writeup_report.md - this file summarizes the results
- run1.mp4 - the video file that capture the automonomos driving on track 1 using training model above


## Model Architecture and Training Strategy

### 1. Solution Design Approach

I first started with Instructor David's suggestion on building a simple Keras model with Flatten and Dense layers to prove a working pipeline. Once it was working, I added LeNet to be the first approval. The first result got the steering angle stucked at 25 degree and car ran off the road into the water or runnining in circle right off the start. I later figured that it was due to typo about fixated the steering angles to always positive when reading from file.

As the LeNet model trained, it was able to stay on track for a short distance before it went off road, but the car was still not capable of entering corner. I followed up with normalizing the training data set by dividing the dataset with 255 and minus 0.5. I also added a layer to crop out the noise above the road and the portion of image below the hood of the car. This helped the model to train faster as less unnecessary data to process. The improvement was not too obvious. So I updated the model to use Nvidia convulation model. However, I kept running into memory allocation issue as I memtioned earlier. As a result, I adjusted layers size with smalled output layers and using MaxPooling and dropout layers to further reduced the network size to compensate the memory allocation issue. 

However, this was still not enough to improve the result significantly. I started looking into the dataset and tried to balance the dataset by looking at their distributions to trim down the excessive data points. See section Data Preprocessing for detail. 

After all the improvement above, the final model was able to train the car to drive itself, and enter and exit corners. I then recorded the vehicle recovering from the left side and right sides of the road back to center. Please see run1.mpg4 for detail. 

### 2. An appropriate model architecture has been employed

My final model consists of 3 parts: data preprocessor, data generator, and a nvidia convnet model pipeline. 

#### date preprocessing

Recorded data are not necessary all useful data. Some times data of certain category may be multiple times many more then the other categories, which will lead to the trained model to become bias to the label of the majority. In this project, car appears to run striaght on open road with 0 steering angles more than making turns, which leads to heavy presents of 0 steering angle data than positive and negative steering data. The distribution below shows the imbalance of the dataset. Our goal is to preprocess the dataset, so that it has a health distribution of every data labels or steering angles to avoid overfitting.

[//]: # (Image References)

 <img src="./writeup images/Steering angle distribution.png" /><p style="text-align: center;"> Data Visualization </p>

From the distribution, it showed a spike at zero, which has 4000+ samples on 0 steering angle. This represents slightly more than half of the total dataset used. Between +/- 0.25 to 0 has a normal distribution. The remaining of steering angles shows a plateau at both edges on the side. To make it easier, I separated the samples into 5 bins: 
- samples_00 - zero steering angles,
- samples_02 - zero-to-pos_0.5 steering angles
- samples_12 - neg_0.5_to_zero steering angels
- samples_99 - remaining steering angeles outside of the above three 

After a few rounds of trying different drop out ratio, I finalized to use the following drop out, and provided a more reasonable distribution of data lables for training. I shrinks the dataset from 8000 data labels of steering angles to about 1664 useful data labels. The dataset can be futher improve to become a balance dataset. Given that the final result is satisfactory, I stopped balancing the dataset.

[//]: # (Image References)

 <img src="./writeup images/Data preprocess drop ratio.png" /><p style="text-align: center;"> Code snippet </p>
[//]: # (Image References)

 <img src="./writeup images/Improved steering angle distribution.png" /><p style="text-align: center;"> Data Visualization </p>



#### data generator

Taking the suggestion from Instructor David, I am using center, left and right images, with left and right images adding and subtracting a small correction factor to the steering angles to create another set of data for training. I also use the flipped center image to create a mirror image. This increase the improved 1664 data labels by 4 times to 6656 data labels for training and validation dataset. This is a large dataset to feed through the convoulation network. To improve the efficiency, I used generator to yield data at each batch size to avoid loading all data to memoary at once.

[//]: # (Image References)

 <img src="./writeup images/Additional data processing.png" /><p style="text-align: center;"> Code snippet </p>


#### Nvidia Convulation Network

Nvidia convnet is my final choice of network use as it yield better result than other previous choice. This version of is not exactly the same Nvidia convnet from the paper. Due training, even I uses data generateors to yield batch size training data at a time, it still hawks a lot of memory and eventually crashed due to not enough memory allocation. As a result, I had to shrink the size of the first five convulation layers from their original size of 24, 32, 48, 64 and 64 to 16, 24, 32, 48, and 64 with inserting 2 Max Pooling layers to reduce the network size. Plus, I drop the originally 1000 outputs from the Dense layer after Flatten layer to 256 outputs instead. The result is obvious and the car was able to steer left and right to keep itself in the center of the road as well as making turns at the corners. 

[//]: # (Image References)

 <img src="./writeup images/Model pipeline.png" /><p style="text-align: center;">  Code snippet </p>


### 3. Attempts to reduce overfitting in the model

The model contains two dropout layers in order to combat overfitting. 

The model was trained and validated on sample data sets with 80/20 split to ensure that the model was not overfitting. The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

However, the history plot still shows validation loss bounce upward as training loss continues to improve on the 3rd epoch. During simulation, it showed the car tended to steer left and right even it was running on a striaght road. This suggested that I may have dropped to much of steering angle 0 data as the model bias on left and right steering and resulting the car crash due to steering too much. 

[//]: # (Image References)

 <img src="./writeup images/validation_loss_overfitting.png" /><p style="text-align: center;"> Loss Visualization </p>
[//]: # (Image References)

 <img src="./writeup images/3 Epochs overfitting result in crash.png" /><p style="text-align: center;"> 3 epochs unsuccessful attempt </p>

I tried adjust the parameter DROP_ZERO_RATIO to drop less zero steering angles from samples and also not to include left and right images to validation samples as their steering angles are manipulated instead of authentic. This result was worse than before the result.

[//]: # (Image References)

 <img src="./writeup images/validation_loss_overfitting2.png" /><p style="text-align: center;"> Loss Visualization </p>

Since it overfitting happened in 3 epochs, I trained the model with just 1 epoch and oberserved the best result. The trained model was able to finish a lap without going over the side lane marks. Unfortunately, the model came to a crash after 2.5 laps. More improvement can be done by finding the right balance on the dataset, or using 2 epochs, or adding more data collection for training. I will left this to future improvement. 

[//]: # (Image References)

 <video src="./run1.mp4" /><p style="text-align: center;"> Video </p>


### 4. Model parameter tuning

The model used an adam optimizer, so the learning rate was not tuned manually.


### 5. Appropriate training data

Training data was chosen so that the steering angles data is less imbalance than the original dataset. Please see section data_propressing for details about how I created the training data.



